# Домашнее задание. Нейросетевая классификация текстов

В этом домашнем задании вам предстоит самостоятельно решить задачу классификации текстов. Вы будете классифицировать тксты писателей по авторству. Всего 8 разных авторов.

Скачаем датасет:

In [ ]:
!wget -O train_authors.csv https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/train_authors_new.csv

--2024-03-31 20:44:01--  https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/train_authors_new.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5163531 (4.9M) [text/plain]
Saving to: ‘train_authors.csv’

train_authors.csv   100%[===================>]   4.92M  --.-KB/s    in 0.02s   

2024-03-31 20:44:02 (200 MB/s) - ‘train_authors.csv’ saved [5163531/5163531]



In [ ]:
!wget -O test_authors.csv https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/test_authors_new.csv

--2024-03-31 20:44:09--  https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/test_authors_new.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 961703 (939K) [text/plain]
Saving to: ‘test_authors.csv’

test_authors.csv    100%[===================>] 939.16K  --.-KB/s    in 0.005s  

2024-03-31 20:44:10 (195 MB/s) - ‘test_authors.csv’ saved [961703/961703]



Импортируем необходимые библиотеки:

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import nltk
from tqdm import tqdm_notebook
import torch.nn as nn                                                             # тут все блоки нейронных сетей, слои
import torch.nn.functional as F

from collections import Counter
from typing import List
import string

import seaborn
seaborn.set(palette='summer')

# скачиваем нужный пакет данных для работы библиотеки nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Подготовка данных

Загрузим даатсет и посмотрим на данные:

In [ ]:
# Загрузим датасет
import pandas as pd
train_data = pd.read_csv('train_authors.csv')
train_data.head()

,text,label
0,"-Да, я поторопился. Капитан, примите мои извин...",Pratchett
1,-Похороны по первому разряду! Довольно благоро...,Remark
2,"Третий округ штата Мэн настолько велик, что ег...",King
3,В мире существуют миллиарды и миллиарды богов....,Pratchett
4,Особенность историографии киевского периода со...,Akunin


Здесь поле 'text' — это текст, а 'label' — автор текста. 'label' — это целевая переменная, которую мы будем предсказывать. Обратите внимания, что в тестовых данных столбца 'label' нет, вам нужно будет предсказать его значения и отправить в качестве ответа на Яндекс.Контест.

In [ ]:
test_data = pd.read_csv('test_authors.csv')
test_data.head()

,text
0,"-Да, я поторопился. Капитан, примите мои извин..."
1,-Похороны по первому разряду! Довольно благоро...
2,"Третий округ штата Мэн настолько велик, что ег..."
3,В мире существуют миллиарды и миллиарды богов....
4,Особенность историографии киевского периода со...


Заведем словарь соответствия имени автора его номеру:

In [ ]:
writers = ['Akunin', 'Bulychev', 'Chehov', 'Dostoevsky', 'Gogol', 'King',
       'Pratchett', 'Remark']
writers_to_label = {writer: i for i, writer in enumerate(writers)}
label_to_writers = {i: writer for i, writer in enumerate(writers)}

Создадим словарь dataset, который будет устроен так же, как переменная dataset в ноутбуке занятия. В тестовой части (dataset['test']) зададим для удобства всем текстам label=0.

In [ ]:
dataset = {}

dataset['train'] = [{'text':text, 'label':writers_to_label[label]} \
              for text, label in zip(np.array(train_data['text']), np.array(train_data['label']))]
dataset['test'] = [{'text':text, 'label': 0} \
              for text in np.array(test_data['text'])]

Ваша задача — обучить RNN-модель на тренировочных данных и получить максимальное возможное accuracy на тестовой части данных. За основу можно взять код с занятия. **Обратите внимание, что здесь у нас задача классификации на 8 классов, а не на 2, как было на занятии.**

Чтобы улучшить качество базовой модели, можно попробовать различные идеи экспериментов. Вот несколько идей:
* **Модель RNN**. Попробуйте LSTM и GRU. Мы советуем обратить внимание на [GRU](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html), так как интерфейс этого класса ничем не отличается от обычной Vanilla RNN, которую мы использовали на семинаре.
* **Увеличение количества рекуррентных слоев модели**. Это можно сделать с помощью параметра `num_layers` в классе `nn.RNN`. В такой модели выходы первой RNN передаются в качестве входов второй RNN и так далее.
* **Изменение вида агрреграции выхода RNN**. В семинаре мы пробовали аггрегировать выходы RNN с помощью max, mean и получения последнего выхода. Можно попробовать другие варианты. Например, конкатенировать результат агрегации и эмбеддинг с последнего токена.
* **Подбор гиперпараметров и обучение до сходимости**. Можно, например, увеличить количество эпох обучения нейросети, а также попробовать различные гиперпараметры: размер словаря, `dropout_rate`, `hidden_dim`.

## Сдача задания

Ниже приведена функция, которую вам необходимо запустить для обученной модели, чтобы получить предсказания на тестовой выборке. Здесь model — ваша обученная модель, dataloader — test_dataloader, построенный на основе тестовой части данных (dataset['test']):

In [ ]:
def get_predictions(model, dataloader):
    """
    Calculate accuracy on data from dataloader.
    """

    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in tqdm_notebook(dataloader,
                                   desc=f'Evaluating'):
            logits = model(batch['input_ids'])
            predictions.append(logits.argmax(dim=1))

    predictions = torch.cat(predictions).data.cpu().numpy()

    return predictions

In [ ]:
predictions = get_predictions(model, test_dataloader)
predictions = [label_to_writers[x] for x in predictions]
predictions

In [ ]:
np.save('submission_hw07.npy', predictions, allow_pickle=True)
print('Ответ сохранен в файл `submission_hw07.npy`')

Ответ сохранен в файл `submission_hw07.npy`
